In [1]:
from dotenv import load_dotenv

load_dotenv()

True

# Pinecone

Pinecone은 고성능 벡터 데이터베이스로, AI 및 머신러닝 애플리케이션을 위한 효율적인 벡터 저장 및 검색 솔루션입니다. 

**주요 특징:**

1. 벡터 데이터 저장 및 검색: 고차원 벡터 데이터를 효율적으로 저장하고 검색할 수 있는 시스템을 제공합니다.

2. 확장성: 대규모 데이터셋을 처리할 수 있는 확장 가능한 아키텍처를 갖추고 있습니다.

3. 개발자 친화적: API를 통해 쉽게 사용할 수 있으며, 다양한 AI 모델과 호환됩니다.

4. 완전 관리형 서비스: 사용자가 인프라 관리에 신경 쓰지 않고 애플리케이션 개발에 집중할 수 있습니다.

**주요 장점:**

1. 고성능 검색: 수십억 개의 아이템을 밀리초 단위로 빠르게 검색할 수 있습니다.

2. AI 애플리케이션 강화: 관련 정보 검색에 의존하는 AI 애플리케이션의 성능을 향상시킬 수 있습니다.

3. 쉬운 시작 및 확장: 몇 번의 클릭이나 API 호출만으로 계정 생성과 인덱스 설정이 가능합니다.

**참고**

- [Pinecone 공식 홈페이지](https://docs.pinecone.io/integrations/langchain)
- [Pinecone 랭체인](https://python.langchain.com/v0.2/docs/integrations/vectorstores/pinecone/)

## 설치

In [ ]:
# 주석 해제 후 설치
# !pip install -U langchain-teddynote

한글 불용어 사전 가져오기 (추후 토크나이저에 사용)

In [2]:
from langchain_teddynote.korean import stopwords

# 한글 불용어 사전 불러오기 (불용어 사전 출처: https://www.ranks.nl/stopwords/korean)
stopwords()[:20]

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다']

## 데이터 전처리

아래는 일반 문서의 전처리 과정입니다. `ROOT_DIR` 하위에 있는 모든 `.pdf` 파일을 읽어와 `document_lsit` 에 저장합니다.

In [3]:
import glob
from langchain.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


ROOT_DIR = "./data"
files = glob.glob(ROOT_DIR + "/*.pdf")

document_list = []
for file in files:
    docs = PyMuPDFLoader(file).load()
    document_list.extend(docs)

Chunk 크기로 나눈 뒤, 분할합니다.

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=350, chunk_overlap=50)
split_docs = text_splitter.split_documents(document_list)

Pinecone 에 DB 저장하기 위한 문서 전처리를 수행합니다. 이 과정에서 `metadata_keys` 를 지정할 수 있습니다.

추가로 metadata 를 태깅하고 싶은 경우 사전 처리 작업에서 미리 metadata 를 추가한 뒤 진행합니다.

In [ ]:
from langchain_teddynote.community.pinecone import preprocess_documents

contents, metadatas = preprocess_documents(
    split_docs=split_docs, metadata_keys=["source", "page"]
)

## Pinecone: 인덱스 생성

파인콘의 새로운 인덱스를 생성합니다.

![pinecone-01.png](./images/pinecone-01.png)

In [6]:
import os
from langchain_teddynote.community.pinecone import create_index

pc_index = create_index(
    api_key=os.environ["PINECONE_API_KEY"],
    index_name="teddynote-db-index",  # 인덱스 이름을 지정합니다.
    dimension=4096,  # Embedding 차원과 맞춥니다. (OpenAIEmbeddings: 1536, UpstageEmbeddings: 4096)
    metric="dotproduct",  # 유사도 측정 방법을 지정합니다. (dotproduct, euclidean, cosine)
)

[create_index]
{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}


Sparse Encoder 를 생성합니다. 여기서는 Kiwi 토크나이저를 사용한 Sparse Encoder 를 생성합니다.

In [ ]:
from langchain_teddynote.community.pinecone import (
    create_sparse_encoder,
    fit_save_sparse_encoder,
)

# 한글 불용어 사전 + Kiwi 형태소 분석기를 사용합니다.
sparse_encoder = create_sparse_encoder(stopwords(), mode="kiwi")

Sparse Encoder 에 Corpus 를 학습합니다.

- `save_path`: Sparse Encoder 를 저장할 경로입니다. 추후에 `pickle` 형식으로 저장한 Sparse Encoder 를 불러와 Query 임베딩할 때 사용합니다. 따라서, 이를 저장할 경로를 지정합니다.

In [8]:
# Sparse Encoder 를 학습합니다.
# 저장하는 경로(save_path)를 잘 기억합니다.
saved_path = fit_save_sparse_encoder(
    sparse_encoder=sparse_encoder, contents=contents, save_path="./sparse_encoder.pkl"
)

  0%|          | 0/160 [00:00<?, ?it/s]

[fit_save_sparse_encoder] Saved sparse encoder to ./sparse_encoder.pkl


### Pinecone: DB Index에 추가 (Upsert)

![](./images/pinecone-02.png)

- `context`: 문서의 내용입니다.
- `page`: 문서의 페이지 번호입니다.
- `source`: 문서의 출처입니다.
- `values`: Embedder 를 통해 얻은 문서의 임베딩입니다.
- `sparse values`: Sparse Encoder 를 통해 얻은 문서의 임베딩입니다.

In [9]:
from langchain_teddynote.community.pinecone import upsert_documents
from langchain_upstage import UpstageEmbeddings


upsert_documents(
    index=pc_index,  # Pinecone 인덱스
    namespace="teddynote-namespace-01",  # Pinecone namespace
    contents=contents,  # 이전에 전처리한 문서 내용
    metadatas=metadatas,  # 이전에 전처리한 문서 메타데이터
    sparse_encoder=sparse_encoder,  # Sparse encoder
    embedder=UpstageEmbeddings(
        model="solar-embedding-1-large-passage"
    ),  # Dense Embedder
)

  0%|          | 0/10 [00:00<?, ?it/s]

[upsert_documents]
{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'teddynote-namespace-01': {'vector_count': 160}},
 'total_vector_count': 160}


## 조회

In [13]:
from langchain_teddynote.community.pinecone import init_pinecone_index

pinecone_params = init_pinecone_index(
    index_name="teddynote-db-index",  # Pinecone 인덱스 이름
    namespace="teddynote-namespace-01",  # Pinecone Namespace
    api_key=os.environ["PINECONE_API_KEY"],  # Pinecone API Key
    sparse_encoder_pkl_path="./sparse_encoder.pkl",  # Sparse Encoder 저장경로(save_path)
    stopwords=stopwords(),  # 불용어 사전
    tokenizer="kiwi",
    embeddings=UpstageEmbeddings(
        model="solar-embedding-1-large-query"
    ),  # Dense Embedder
    top_k=3,  # Top-K 문서 반환 개수
    alpha=0.5,  # alpha=0.75로 설정한 경우, (0.75: Dense Embedding, 0.25: Sparse Embedding)
)

[init_pinecone_index]
{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'teddynote-namespace-01': {'vector_count': 160}},
 'total_vector_count': 160}


`PineconeKiwiHybridRetriever` 를 생성합니다.

In [14]:
from langchain_teddynote.community.pinecone import PineconeKiwiHybridRetriever

pinecone_retriever = PineconeKiwiHybridRetriever(**pinecone_params)

In [18]:
# 실행 결과
search_results = pinecone_retriever.invoke("앤스로픽")
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

namespace teddynote-namespace-01
1. 정책/법제  
2. 기업/산업 
3. 기술/연구 
 4. 인력/교육
구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 
n 구글이 앤스로픽에 최대 20억 달러 투자에 합의하고 5억 달러를 우선 투자했으며, 앤스로픽은 
구글과 클라우드 서비스 사용 계약도 체결
n 3대 클라우드 사업자인 구글, 마이크로소프트, 아마존은 차세대 AI 모델의 대표 기업인 
앤스로픽 및 오픈AI와 협력을 확대하는 추세
KEY Contents
£ 구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공
n 구글이 2023년 10월 27일 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억
{'source': './data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 13.0, 'score': 0.56979203}


달러를 우선 투자하고 향후 15억 달러를 추가로 투자할 방침
∙구글은 2023년 2월 앤스로픽에 이미 5억 5,000만 달러를 투자한 바 있으며, 아마존도 지난 9월 
앤스로픽에 최대 40억 달러의 투자 계획을 공개
∙한편, 2023년 11월 8일 블룸버그 보도에 따르면 앤스로픽은 구글의 클라우드 서비스 사용을 위해 
4년간 30억 달러 규모의 계약을 체결
∙오픈AI 창업자 그룹의 일원이었던 다리오(Dario Amodei)와 다니엘라 아모데이(Daniela Amodei) 
남매가 2021년 설립한 앤스로픽은 챗GPT의 대항마 ‘클로드(Claude)’ LLM을 개발
{'source': './data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 13.0, 'score': 0.46733376}


▹ 알리바바 클라우드, 최신 LLM ‘통이치엔원 2.0’ 공개 ······················································ 9
   ▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가